<a href="https://colab.research.google.com/github/ProfessorPatrickSlatraigh/CST3512/blob/main/CST3512_Folium_NYPD_StopFrisk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Folium Choropleth Map of NYPD StopFrisk 2020 Data by ZIP Code    

**originally by Louis Casanave**     
*Louis is a Data Scientist from Brooklyn, NY*     

['Using Folium on Police Data: Say YES to the MESS with Choropleths' on Medium](https://towardsdatascience.com/using-folium-on-police-data-3207e505c649) -- *Louis's original article uses 2020 data*.  
    

##Housekeeping    

###Import the usual suspects    

In [2]:
import pandas as pd
import folium

###Load data files to current working directory    

In [ ]:
# load geojson file of NYC map areas from Louis Casanove Github raw file
!curl https://raw.githubusercontent.com/ProfessorPatrickSlatraigh/geo/main/nyc-zip-code-tabulation-areas-polygons.geojson -o "nyc-zip-code-tabulation-areas-polygons.geojson"

In [ ]:
# the following `curl` statement may be used to pull a local copy of the 2021 data file
!curl https://raw.githubusercontent.com/ProfessorPatrickSlatraigh/data/main/nypd_StopFrisk_2020.csv -o "nypd_StopFrisk_2020.csv"

##Data Wrangling   

###Create a `sf_df` DataFrame for the source stop and frisk data    

In [5]:
sf_df = pd.read_csv('/content/nypd_StopFrisk_2020.csv')

####Inspect the initial data    

In [ ]:
# saving the shape so we can use it later 
sf_shape = sf_df.shape
sf_shape

In [ ]:
# taking a look at what columns we have in the data
sf_cols = sf_df.columns
sf_cols

###Keep only the necessary data in a new `zip_code_frequency_df` DataFrame

In [8]:
zipcode_frequency_df = sf_df.drop(columns = ['STOP_FRISK_DATE', 'STOP_FRISK_TIME', 'YEAR2', 'MONTH2',
       'DAY2', 'STOP_WAS_INITIATED', 'RECORD_STATUS_CODE',
       'ISSUING_OFFICER_RANK', 'ISSUING_OFFICER_COMMAND_CODE',
       'SUPERVISING_OFFICER_RANK', 'SUPERVISING_OFFICER_COMMAND_CODE',
       'LOCATION_IN_OUT_CODE', 'JURISDICTION_CODE', 'JURISDICTION_DESCRIPTION',
       'OBSERVED_DURATION_MINUTES', 'SUSPECTED_CRIME_DESCRIPTION',
       'STOP_DURATION_MINUTES', 'OFFICER_EXPLAINED_STOP_FLAG',
       'OFFICER_NOT_EXPLAINED_STOP_DESCRIPTION', 'OTHER_PERSON_STOPPED_FLAG',
       'SUSPECT_ARRESTED_FLAG', 'SUSPECT_ARREST_OFFENSE',
       'SUMMONS_ISSUED_FLAG', 'SUMMONS_OFFENSE_DESCRIPTION',
       'OFFICER_IN_UNIFORM_FLAG', 'ID_CARD_IDENTIFIES_OFFICER_FLAG',
       'SHIELD_IDENTIFIES_OFFICER_FLAG', 'VERBAL_IDENTIFIES_OFFICER_FLAG',
       'FRISKED_FLAG', 'SEARCHED_FLAG', 'ASK_FOR_CONSENT_FLG',
       'CONSENT_GIVEN_FLG', 'OTHER_CONTRABAND_FLAG', 'FIREARM_FLAG',
       'KNIFE_CUTTER_FLAG', 'OTHER_WEAPON_FLAG', 'WEAPON_FOUND_FLAG',
       'PHYSICAL_FORCE_CEW_FLAG', 'PHYSICAL_FORCE_DRAW_POINT_FIREARM_FLAG',
       'PHYSICAL_FORCE_HANDCUFF_SUSPECT_FLAG',
       'PHYSICAL_FORCE_OC_SPRAY_USED_FLAG', 'PHYSICAL_FORCE_OTHER_FLAG',
       'PHYSICAL_FORCE_RESTRAINT_USED_FLAG',
       'PHYSICAL_FORCE_VERBAL_INSTRUCTION_FLAG',
       'PHYSICAL_FORCE_WEAPON_IMPACT_FLAG',
       'BACKROUND_CIRCUMSTANCES_VIOLENT_CRIME_FLAG',
       'BACKROUND_CIRCUMSTANCES_SUSPECT_KNOWN_TO_CARRY_WEAPON_FLAG',
       'SUSPECTS_ACTIONS_CASING_FLAG',
       'SUSPECTS_ACTIONS_CONCEALED_POSSESSION_WEAPON_FLAG',
       'SUSPECTS_ACTIONS_DECRIPTION_FLAG',
       'SUSPECTS_ACTIONS_DRUG_TRANSACTIONS_FLAG',
       'SUSPECTS_ACTIONS_IDENTIFY_CRIME_PATTERN_FLAG',
       'SUSPECTS_ACTIONS_LOOKOUT_FLAG', 'SUSPECTS_ACTIONS_OTHER_FLAG',
       'SUSPECTS_ACTIONS_PROXIMITY_TO_SCENE_FLAG',
       'SEARCH_BASIS_ADMISSION_FLAG', 'SEARCH_BASIS_CONSENT_FLAG',
       'SEARCH_BASIS_HARD_OBJECT_FLAG',
       'SEARCH_BASIS_INCIDENTAL_TO_ARREST_FLAG', 'SEARCH_BASIS_OTHER_FLAG',
       'SEARCH_BASIS_OUTLINE_FLAG', 'DEMEANOR_CODE',
       'DEMEANOR_OF_PERSON_STOPPED', 'SUSPECT_REPORTED_AGE', 'SUSPECT_SEX',
       'SUSPECT_RACE_DESCRIPTION', 'SUSPECT_HEIGHT', 'SUSPECT_WEIGHT',
       'SUSPECT_BODY_BUILD_TYPE', 'SUSPECT_EYE_COLOR', 'SUSPECT_HAIR_COLOR',
       'SUSPECT_OTHER_DESCRIPTION', 'STOP_LOCATION_PRECINCT',
       'STOP_LOCATION_SECTOR_CODE', 'STOP_LOCATION_APARTMENT',
        'STOP_LOCATION_X', 'STOP_LOCATION_Y',
       'STOP_LOCATION_FULL_ADDRESS', 'STOP_LOCATION_STREET_NAME',
       'STOP_LOCATION_PATROL_BORO_NAME', 'STOP_LOCATION_BORO_NAME']) 


###Find and Transform Missing Data    

In [ ]:
# looking for null values
zipcode_frequency_df.info()

In [ ]:
# found the null values as "(null)" in dataframe
zipcode_frequency_df['STOP_LOCATION_ZIP_CODE'].describe()

In [ ]:
# displaying the percent of zipcode data missing, rounded to the 0.00% 
percent_missing = zipcode_frequency_df['STOP_LOCATION_ZIP_CODE'].describe()[3]/sf_shape[0]
print(f' This data has {percent_missing:.2f} percent of its zipcode data missing.')

###Data Aggregation   

In [ ]:
# this will change all stop ids to 1 for aggregation
zipcode_frequency_df['STOP_ID'] = 1

# this will aggregate our data
zipcode_frequency_df = zipcode_frequency_df.groupby("STOP_LOCATION_ZIP_CODE").count()

# now we can see the dataframe as it is now
zipcode_frequency_df.head()

###Additional Data Transformations     
 - rename columns
 - transformation values to strings for mapping 
 - delete null values    

In [13]:
# this will rename our column for clarity and sanity 
zipcode_frequency_df = zipcode_frequency_df.rename(columns = {'STOP_ID' : 'FREQUENCY'})

# making the zipcode information into a string so the map doesn't break
zipcode_frequency_df.index = zipcode_frequency_df.index.astype(str)

# this will get rid of those null values that we decided to delete and keep track of
zipcode_frequency_df = zipcode_frequency_df.loc[zipcode_frequency_df.index != '(null)']

##Review Data Before Generating a Map    

In [ ]:
print(zipcode_frequency_df.head())
print(zipcode_frequency_df.tail())

##Generate a Choropleth Map    

*note: there are several changes required since the original article and code.*    

In [ ]:
# making the base map of nyc
## the following original statement blows up on kwarg `default_zoom_start`     ##
# map = folium.Map(location=[40.693943, -73.985880], default_zoom_start=15)
## replaced the Map object creator above with the line below as of Nov-2022    ##
map = folium.Map(location=[40.695000, -74.000000], zoom_start=10, min_zoom=10, control_scale=True)

# making the base map of nyc

# taking the geojson zipcode file, assigning data to my dataframe, 
# feeding index and instances into columns, key_on zipcode within properties 
# within feature, setting the color and legend name

## the following .choropleth() method is has bee deprecated in Folium          ##
## the following code is in Luis's original example, but is replaced w/GeoJson ##  
# map.choropleth(geo_data="nyc-zip-code-tabulation-areas-polygons.geojson",
#                data=zipcode_frequency_df,
#                columns=[zipcode_frequency_df.index, 'FREQUENCY'],
#                key_on='feature.properties.postalCode', 
#                fill_color='YlOrBr', fill_opacity=0.7, line_opacity=0.2,
#                legend_name='Frequency')
## the deprecated code block is above -- following code is current at Nov-2022 ##

# use `.Choropleth()` instead of `.choropleth()`

zip_geos = folium.Choropleth(
    geo_data="nyc-zip-code-tabulation-areas-polygons.geojson",  # shape of ZIPs
    name='choropleth',   
    data=zipcode_frequency_df, 
    columns=[zipcode_frequency_df.index, 'FREQUENCY'],    # by ZIP using FREQUENCY
    key_on='feature.properties.postalCode', # feature.properties.postalCode is ZIP
    fill_color='YlOrBr',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color = "White",  # fill in ZIPs with missing data using White  
    legend_name='Stop Rate').add_to(map)

# folium.GeoJsonTooltip(fields=["postalCode"]).add_to(zip_geos)

folium.LayerControl().add_to(map)    # adding layer control  

# Display ZIP Label
zip_geos.geojson.add_child(
    folium.features.GeoJsonTooltip(['postalCode'], labels=False))  # tooltip ZIP

map



---



Want to practice additional Folium choropleth map skills?    
Read ['How to step up your Folium Choropleth Map skills' in TowardDatascience.com](https://towardsdatascience.com/how-to-step-up-your-folium-choropleth-map-skills-17cf6de7c6fe)

---